Import packages

In [ ]:
from __future__ import print_function
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell
allow_stdin=True

In [ ]:
import sys,os,argparse
from IPython.display import HTML
CONFIG_FILE = '.config_ipynb'
if os.path.isfile(CONFIG_FILE):
    with open(CONFIG_FILE) as f:
        sys.argv = f.read().split()

parser = argparse.ArgumentParser()
parser.add_argument("input_file",help="Input image, directory, or npy.")
parser.add_argument("--int_param", type=str, default="mnist", help="dataset, mnist or fashion")
args = parser.parse_args()
p = args.int_param
print(args.input_file,p)

read the dataset

In [ ]:
mydataset = p
flag = True
while flag:
#     mydataset = input("Please choose the dataset to be imported. The options are mnist & fashion: You chose -> ")
    if mydataset == "mnist":
        from keras.datasets import mnist
        flag = False
    elif mydataset == "fashion":
        from keras.datasets import fashion_mnist
        flag = False
    else:
        print("You did not choose a right dataset to import, maybe next time!")

The following functions are letting the main code to see the other notebooks in the same directory (train, test, model, output)

In [ ]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [ ]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [ ]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [ ]:
sys.meta_path.append(NotebookFinder())

Set the variables

In [ ]:
batch_size = 128
num_classes = 10
epochs = 20
%store num_classes
%store epochs
%store batch_size

In [ ]:
if mydataset == "mnist":
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
elif mydataset == "fashion":
    (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
%store x_train
%store y_train
%store x_test
%store y_test

Train the model by calling the train.ipynb notebook

In [ ]:
%run ./train.ipynb

Test the model by calling the test.ipynb notebook

In [ ]:
%run ./test.ipynb
%store -r test_score

In [ ]:
%run ./output.ipynb

In [ ]:
print("Finished!")